In [22]:
%load_ext autoreload
%autoreload 2  
%reload_ext autoreload
import sys
sys.path.append('..')
import pickle
%load_ext autoreload
%autoreload 2  
%reload_ext autoreload
import sys
sys.path.append('..')
import pickle
from pathlib import Path
from rich import print
from rich.console import Console
import pdb
import json
import random
import copy
import time
import re
from ruamel.yaml import YAML 
sys.path.append('/Users/georgiazhou/research_machine/dialop/dialop/notebooks')
from notebooks import strategy_source_function as agent_testing
import importlib
importlib.reload(agent_testing) 
console = Console()

# Import your modules
from dialop.evaluate import CheckpointManager
from dialop.responses_class import ParallelConversations, ResponseManager, Conversation
from dialop.envs.planning import PlanningEnv
from skills import Agent_tell
from dialop.games.planning import PlanningGame
print("Imports successful!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/georgiazhou/research_machine/dialop/.venv/bin/python

Imports successful!

Imports successful!

In [9]:
def wait_for_initial_checkpoint(checkpoint_mgr):
    """Wait for initial checkpoint from evaluate.py"""
    print("Waiting for initial checkpoint from evaluate.py...")
    while True:
        # Look for initial_state checkpoint
        if (checkpoint_mgr.checkpoint_dir / "initial_state.pkl").exists():
            print("Found initial checkpoint!")
            # Load the initial state
            state = checkpoint_mgr.load_full_state("initial_state")
            return state
        time.sleep(1) 

In [28]:
count = 0
def main():
    # Initialize checkpoint manager
    checkpoint_mgr = CheckpointManager()
    
    
    # Wait for initial checkpoint from evaluate.py
    initial_state = wait_for_initial_checkpoint(checkpoint_mgr)
    global count
    # count is the number of times the main function has been run
    
    
    # Initialize parallel conversations with loaded state
 
        
    try:
    # Access key components
        players = initial_state['players']
        extracted_features = initial_state.get('extracted_features', [])
        t = initial_state.get('t', 0)
        env = initial_state.get('env', None)
        agent = initial_state['players']['agent']
        user = initial_state['players']['user']
        if not env:
            print("No environment found in checkpoint.")
        print(f"\nTurn: {t}")
        print(f"Features: {extracted_features}")
        print(f"Players: {list(players.keys())}")
        strategy_response_list = agent_testing.strategy_sampling(env)
    
    except Exception as e:
        print(f"Error loading checkpoint: {str(e)}")
    filepath = "/Users/georgiazhou/research_machine/dialop/dialop/notebooks/RL_data/RL_experiement/experiment8/"
    open(f"{filepath}strategy_response_list_{count}", "w").write(json.dumps(strategy_response_list))
    open(f"{filepath}base_user_{count}", "w").write(user.prompt)
    open(f"{filepath}base_agent_{count}", "w").write(agent.prompt)
    open(f"{filepath}base_action_log_{count}", "w").write(json.dumps(env.game.action_log))
    open(f"{filepath}base_events_{count}", "w").write(json.dumps(env.events))
    
    open(f"{filepath}base_extracted_features_{count}", "w").write(json.dumps(extracted_features))
    
    agent_3_responses = agent_testing.n_different_responses(players, strategy_response_list, t = t)
    open(f"{filepath}base_agent_3_responses_{count}", "w").write(json.dumps(agent_3_responses))
    parallel_convs = agent_testing.initialize_parallel_conversations(agent_3_responses, strategy_response_list, players, env, t)
    style_response = agent_testing.style_judger(parallel_convs)
    parallel_convs = agent_testing.step_parallel_conversations(parallel_convs)
    for  c_idx, conversation in enumerate(parallel_convs.conversations):
        open(f"{filepath}3_paths_agent_{count}_{c_idx}", "w").write(json.dumps(str(conversation.history)))
    user_conversations_no_pref = agent_testing.clean_user_conversation(user.user_prompt_obss, parallel_convs)
    parallel_convs, preferences_only = agent_testing.initialize_10_user_conversations(parallel_convs, user_conversations_no_pref, style_response, extracted_features, reset_sonlist = True)
    open(f"{filepath}10_preferences_only_{count}", "w").write(json.dumps(preferences_only))
    conversation_states = agent_testing.step_all_user_responses( parallel_convs)
    open(f"{filepath}30_conversation_states_{count}", "w").write(json.dumps(str(conversation_states)))
    agent_proposals = agent_testing.run_until_agent_proposals( parallel_convs)
    open(f"{filepath}30_agent_proposals_{count}", "w").write(json.dumps(str(agent_proposals)))
    scores, rating_prompts = agent_testing.api_rate_agent_proposals(agent_proposals, parallel_convs)
    open(f"{filepath}1_example_rating_prompt_{count}", "w").write(json.dumps(str(rating_prompts[0][0])))
    open(f"{filepath}30_scores_{count}", "w").write(json.dumps(str(scores)))
    for i, conv in enumerate(parallel_convs.conversations):
        conv_path = []
        for son_conv in conv.son_convs:
            conv_path.append(son_conv.history)
        open(f"{filepath}10_agent_thinking_paths_after_user_response{count}_{i}", "w").write(json.dumps(str(conv_path)))
    best_path = agent_testing.choose_best_path(scores)
    open(f"{filepath}the_best_path_{count}", "w").write(json.dumps(str(parallel_convs.conversations[best_path].history)))
    checkpoint_name = agent_testing.save_best_path_as_checkpoint(parallel_convs, best_path, checkpoint_mgr)
    print(f"Successfully processed checkpoint. Saved as: {checkpoint_name}")
    count += 1
    return checkpoint_name

    # Rn conversations in parallel
    
    

In [29]:

def continuous_monitoring():
    """Continuously monitor for new checkpoints and process them"""
    print("Starting continuous monitoring...")
    count = 0
    
    while True:
        main()
        
        
        try:
            # Run main process
            
            
            # Clean up initial state checkpoint to prepare for next round
            checkpoint_mgr = CheckpointManager()
            initial_state_path = checkpoint_mgr.checkpoint_dir / "initial_state.pkl"
            if initial_state_path.exists():
                initial_state_path.unlink()  # Delete the initial state file
            
            print("\nWaiting for next checkpoint...")
            time.sleep(5)  # Wait before starting next cycle
            
        except KeyboardInterrupt:
            print("\nStopping monitoring...")
            break
        except Exception as e:
            print(f"\nError in monitoring cycle: {e}")
            time.sleep(5)  # Wait before retrying

# Start the continuous monitoring
if __name__ == "__main__":
    continuous_monitoring()
# Load your saved checkpoint


Starting continuous monitoring...

Waiting for initial checkpoint from evaluate.py...

Found initial checkpoint!

Turn: 6

Features: {'AtLeastOneEventType': ['landmark', 'High'], 'WantToGo': ['The Mall', 'Unknown'], 'has_parking': ['not a
deal breaker', 'Low']}

...

Response:  [Central Plaza, The Mall, The Tower]

stop:  stop

CompletionUsage(
    completion_tokens=10,
    prompt_tokens=2812,
    total_tokens=2822,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

Final proposal: [Central Plaza, The Mall, The Tower]

IndexError: list index out of range

In [ ]:
   #print(user.prompt)
    